In [ ]:
import re
from datetime import datetime
from calendar_ipynb.ipywidgets.calendar_selection import (
    select_calendars,
    get_selected_calendars,
)
from calendar_ipynb.events import fetch_events_parallel

EMAIL_IDS = ["fahimalizain@gmail.com", "fahim.zain@kwalee.com"]
select_calendars(EMAIL_IDS)

In [86]:
from calendar_ipynb.events import get_calendar_service


TITLE_REGEX = r"^(Clean Black Tank \| FAAZ) $"
RENAME_REGEX = r"\1"

from_datetime = datetime.fromisoformat("2025-01-01T00:00:00Z".replace("Z", "+00:00"))
to_datetime = datetime.fromisoformat("2026-01-01T23:59:59Z".replace("Z", "+00:00"))
print(get_selected_calendars())

all_events = fetch_events_parallel(
    email_map=get_selected_calendars(),
    from_datetime=from_datetime,
    to_datetime=to_datetime
)

matcher = re.compile(TITLE_REGEX)
EVENTS_TO_RENAME = []
for event in all_events:
    if not matcher.match(event['summary']):
        continue

    if event.get("status", None) != "confirmed":
        continue

    if event.get("eventType", None) not in ["default", "fromGmail"]:
        continue

    if "creator" in event and event["creator"].get("email", None) != event["email"]:
        continue

    EVENTS_TO_RENAME.append(event)

print(f"Found {len(EVENTS_TO_RENAME)} events to rename")
unique_summaries = list(set(event['summary'] for event in EVENTS_TO_RENAME))
for summary in unique_summaries:
    print(f" - '{summary}' --> '{re.sub(TITLE_REGEX, RENAME_REGEX, summary)}'")

{'fahimalizain@gmail.com': ['esvag4fbb8q192q3ksipqh5d2g@group.calendar.google.com', 'j9d5885raedlop9apnj5c4odoo@group.calendar.google.com', 'mbn0dk4phhhufr6hpbrl15p4h0@group.calendar.google.com', 'fahimalizain@gmail.com'], 'fahim.zain@kwalee.com': ['fahim.zain@kwalee.com']}


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:calendar_ipynb.events_incremental:Sync completed for fahimalizain@gmail.com/mbn0dk4phhhufr6hpbrl15p4h0@group.calendar.google.com: Added 0, Updated 0, Deleted 0
INFO:calendar_ipynb.events_incremental:Sync completed for fahimalizain@gmail.com/j9d5885raedlop9apnj5c4odoo@group.calendar.google.com: Added 0, Updated 0, Deleted 0
INFO:calendar_ipynb.events_incremental:Sync completed for fahimalizain@gmail.com/fahimalizain@gmail.com: Added 0, Updated 0, Deleted 0
INFO:calendar_ipynb.events_incremental:Sync completed for fahim.zain@kwalee.com/fahim

Found 1 events to rename
 - 'Clean Black Tank | FAAZ ' --> 'Clean Black Tank | FAAZ'


In [87]:
SERVICE_MAP = {
    email: get_calendar_service(email) for email in EMAIL_IDS
}

for event in EVENTS_TO_RENAME:
    original_summary = event['summary']
    new_summary = re.sub(TITLE_REGEX, RENAME_REGEX, original_summary)

    service = SERVICE_MAP[event['email']]
    service.events().update(
        calendarId=event["calendar_id"], eventId=event["id"], body=dict(event, summary=new_summary)
    ).execute()
    print("Renamed", f"'{original_summary}'", "to", f"'{new_summary}'", event["start"])

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


Renamed 'Clean Black Tank | FAAZ ' to 'Clean Black Tank | FAAZ' {'dateTime': '2025-06-19T10:15:00+05:30', 'timeZone': 'Asia/Kolkata'}
